In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract

import os
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
imgPaths=glob('./Selected/*.jpeg')

In [6]:
allBusinessCard=pd.DataFrame(columns=['id','text'])

# for imgPath in imgPaths: 
for imgPath in tqdm(imgPaths,desc='Business Cards'):
    # imgPath=imgPaths[54]
    _ , filename=os.path.split(imgPath)
    # os.path.split(imgPath) returns ('./Selected', '000.jpeg')
    # n Python, _ is a convention that means: “I don’t care about this value.”
    
    #-> Exract data and text
    image=cv2.imread(imgPath)
    data=pytesseract.image_to_data(image)
    # Lambda function receives a list seperates by \n and each element containing multiple \t 
    # So we seperate each element upto a table entry level
    dataList= list(map(lambda x:x.split('\t'),data.split('\n')))
    df=pd.DataFrame(dataList[1:],columns=dataList[0])
    df.dropna(inplace=True)
    df['conf']=df['conf'].astype(float)
    df['conf']=df['conf'].astype(int)
    
    # usefulData=df.query('conf>=30')
    usefulData=df[df['conf'] >= 30]
    
    #DataFrame
    businessCard=pd.DataFrame()
    businessCard['text']=usefulData['text']
    businessCard['id']=filename

    #Concatenation
    allBusinessCard=pd.concat((allBusinessCard,businessCard))


Business Cards: 100%|████████████████████████████████████████████████████████████████| 293/293 [02:34<00:00,  1.90it/s]


In [5]:
allBusinessCard.to_csv('BusinessCard.csv',index=False)